## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,42.48,65,0,1.83,clear sky
1,1,Atuona,PF,-9.8000,-139.0333,78.08,78,23,22.32,few clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.96,83,75,11.50,broken clouds
3,3,Cam Ranh,VN,11.9214,109.1591,91.36,52,0,2.30,clear sky
4,4,Poros,GR,37.4994,23.4536,72.91,64,0,4.94,clear sky
5,5,Khatanga,RU,71.9667,102.5000,53.69,59,100,10.92,overcast clouds
6,6,Busselton,AU,-33.6500,115.3333,63.57,69,100,13.91,overcast clouds
7,7,Carnarvon,AU,-24.8667,113.6333,69.84,50,0,12.01,clear sky
8,8,San Andres,CO,12.5847,-81.7006,82.42,78,40,13.80,thunderstorm
9,9,Chumikan,RU,54.7183,135.3150,48.09,79,24,11.59,few clouds


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 85


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.08,78,23,22.32,few clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.96,83,75,11.50,broken clouds
4,4,Poros,GR,37.4994,23.4536,72.91,64,0,4.94,clear sky
6,6,Busselton,AU,-33.6500,115.3333,63.57,69,100,13.91,overcast clouds
7,7,Carnarvon,AU,-24.8667,113.6333,69.84,50,0,12.01,clear sky
8,8,San Andres,CO,12.5847,-81.7006,82.42,78,40,13.80,thunderstorm
10,10,Daoukro,CI,7.0591,-3.9631,70.86,100,76,2.53,broken clouds
11,11,Poum,NC,-20.2333,164.0167,72.90,58,94,15.48,overcast clouds
12,12,Rikitea,PF,-23.1203,-134.9692,71.24,60,49,13.22,scattered clouds
15,15,Ribeira Grande,PT,38.5167,-28.7000,65.14,88,96,20.78,overcast clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
preferred_cities_df.notnull().sum()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.08,78,23,22.32,few clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.96,83,75,11.50,broken clouds
4,4,Poros,GR,37.4994,23.4536,72.91,64,0,4.94,clear sky
6,6,Busselton,AU,-33.6500,115.3333,63.57,69,100,13.91,overcast clouds
7,7,Carnarvon,AU,-24.8667,113.6333,69.84,50,0,12.01,clear sky
8,8,San Andres,CO,12.5847,-81.7006,82.42,78,40,13.80,thunderstorm
10,10,Daoukro,CI,7.0591,-3.9631,70.86,100,76,2.53,broken clouds
11,11,Poum,NC,-20.2333,164.0167,72.90,58,94,15.48,overcast clouds
12,12,Rikitea,PF,-23.1203,-134.9692,71.24,60,49,13.22,scattered clouds
15,15,Ribeira Grande,PT,38.5167,-28.7000,65.14,88,96,20.78,overcast clouds


In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.08,-9.8000,-139.0333,
2,Vaini,TO,78.96,-21.2000,-175.2000,
4,Poros,GR,72.91,37.4994,23.4536,
6,Busselton,AU,63.57,-33.6500,115.3333,
7,Carnarvon,AU,69.84,-24.8667,113.6333,
8,San Andres,CO,82.42,12.5847,-81.7006,
10,Daoukro,CI,70.86,7.0591,-3.9631,
11,Poum,NC,72.90,-20.2333,164.0167,
12,Rikitea,PF,71.24,-23.1203,-134.9692,
15,Ribeira Grande,PT,65.14,38.5167,-28.7000,


In [28]:
len(hotel_df)

491

In [29]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,78.08,-9.8000,-139.0333,Atuona
2,Vaini,TO,78.96,-21.2000,-175.2000,Nukuhetulu
4,Poros,GR,72.91,37.4994,23.4536,Galatas
6,Busselton,AU,63.57,-33.6500,115.3333,Busselton
7,Carnarvon,AU,69.84,-24.8667,113.6333,Carnarvon
8,San Andres,CO,82.42,12.5847,-81.7006,San Andrés
10,Daoukro,CI,70.86,7.0591,-3.9631,Daoukro
11,Poum,NC,72.90,-20.2333,164.0167,Poum
12,Rikitea,PF,71.24,-23.1203,-134.9692,Rikitea
15,Ribeira Grande,PT,65.14,38.5167,-28.7000,Horta


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))